# How to add Memory to an Agent

This notebook goes over adding memory to an Agent. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:

- [Adding memory to an LLM Chain](adding_memory.html)
- [Custom Agents](../../agents/agents/custom_agent.html)

In order to add a memory to an agent we are going to the the following steps:

1. We are going to create an LLMChain with memory.
2. We are going to use that LLMChain to create a custom Agent.

For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the `ConversationBufferMemory` class.

In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper, SerpAPIWrapper
import dotenv
import langchain
import inspect

dotenv.load_dotenv()

True

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

Notice the usage of the `chat_history` variable in the PromptTemplate, which matches up with the dynamic key name in the ConversationBufferMemory.

In [3]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

We can now construct the LLMChain, with the Memory object, and then create the agent.

In [4]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [5]:
agent_chain.run(input="How many people live in canada?")



> Entering new  chain...
Thought: I need to find out the population of Canada
Action: Search
Action Input: Population of Canada
Observation: Canada is a country in North America. Its ten provinces and three territories extend from the Atlantic Ocean to the Pacific Ocean and northward into the Arctic Ocean, making it the world's second-largest country by total area, with the world's longest coastline.
Thought: I need to find out the exact population of Canada
Action: Search
Action Input: Exact population of Canada
Observation: The current population of Canada is 38,762,492 based on projections of the latest United Nations data. The UN estimates the July 1, 2023 population at 38,781,291.
Thought: I now know the final answer
Final Answer: The current population of Canada is 38,762,492.

> Finished chain.


'The current population of Canada is 38,762,492.'

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [6]:
agent_chain.run(input="what is their national anthem called?")



> Entering new  chain...
Thought: I need to find out what the national anthem of Canada is called.
Action: Search
Action Input: National anthem of Canada
Observation: "O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which words were written by the poet and judge Sir Adolphe-Basile Routhier.
Thought: I now know the final answer.
Final Answer: The national anthem of Canada is "O Canada".

> Finished chain.


'The national anthem of Canada is "O Canada".'

We can see that the agent remembered that the previous question was about Canada, and properly asked Google Search what the name of Canada's national anthem was.

For fun, let's compare this to an agent that does NOT have memory.

In [7]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_without_memory = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [8]:
agent_without_memory.run("How many people live in canada?")



> Entering new  chain...
Thought: I need to find out the population of Canada
Action: Search
Action Input: Population of Canada
Observation: Canada is a country in North America. Its ten provinces and three territories extend from the Atlantic Ocean to the Pacific Ocean and northward into the Arctic Ocean, making it the world's second-largest country by total area, with the world's longest coastline.
Thought: I need to find out the exact population of Canada
Action: Search
Action Input: Exact population of Canada
Observation: The current population of Canada is 38,762,492 based on projections of the latest United Nations data. The UN estimates the July 1, 2023 population at 38,781,291.
Thought: I now know the final answer
Final Answer: The current population of Canada is 38,762,492.

> Finished chain.


'The current population of Canada is 38,762,492.'

In [9]:
agent_without_memory.run("what is their national anthem called?")



> Entering new  chain...
Thought: I should search for the answer
Action: Search
Action Input: national anthem
Observation: "The Star-Spangled Banner" is the national anthem of the United States. The lyrics come from the "Defence of Fort M'Henry", a poem written on September 14, 1814, by 35-year-old lawyer and amateur poet ...
Thought: I now know the final answer
Final Answer: The Star-Spangled Banner is the national anthem of the United States.

> Finished chain.


'The Star-Spangled Banner is the national anthem of the United States.'

In [13]:
!python -m pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
